In [1]:
class Professor:
    def __init__(self, name, email, phone, position):
        self.name = name
        self.email = email
        self.phone = phone
        self.position = position
        
    def __str__(self):
        return self.name + str(self.email) + str(self.phone) + '\n' + str(self.position)

In [2]:
import requests
page = requests.get('https://www.hse.ru/org/persons/')  

print(page.text[:251])

<!DOCTYPE html>
<!-- (c) Art. Lebedev Studio | http://www.artlebedev.com/ -->
<html xmlns:perl="urn:perl" lang="ru"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta nam


### etree

In [3]:
from lxml import etree

root = etree.HTML(page.content)
   
print(root.tag) 

html


In [4]:
posts = root[1][1][3][2][1][0][2][1]
print(posts.attrib, posts.tag)

{'class': 'persons__section js-persons-section current', 'count': '272'} div


In [5]:
def phones(post):
    phone = []
    for i in post[0]:
        if i.tag == 'span':
            ph = i.text
            phone.append(ph.strip())
    return phone

In [6]:
def positions(post):
    positions = {}
    for i in post:
        position = ''
        if i.tag == 'span':
            position = str((i.text).strip())
            p = []
            for elem in i:
                if elem.tag == 'a':
                    p.append((elem.text).strip())
            if position == '':
                position = str((i[len(i)-1].tail).strip())
        positions[position] = ' '.join(p)
    return positions

In [7]:
def emails(post):
    mails = []
    for i in post[0]:
        if i.tag == 'a':
            m = i.attrib['data-at']
            mail = (m.replace("-at-", "@")).split('"')[1::2]
            mails.append("".join(mail))
    return mails

In [8]:
def teachers_with_etree(posts):
    profs = []
    for post in posts:
        if 'post person' in post.attrib['class']:
            p = post[0]
            k = 1 if 'main' not in p[0].attrib['class'] else 0
            name = (p[k][0][0][0].tail).strip()
            pos = positions(p[k][0][1])   
            prof = Professor(name, emails(p), phones(p), pos)
            profs.append(prof)
    return profs

In [9]:
profs = teachers_with_etree(posts)

for p in profs:
    print(p)

Абанкина Ирина Всеволодовна['abankinai@hse.ru']['+7(495) 772-9590 *22073']
{'': 'Институт образования Департамент образовательных программ', 'Директор:': 'Институт образования Институт развития образования', 'Профессор:': 'Институт образования Департамент образовательных программ'}
Абанкина Татьяна Всеволодовна['abankinat@hse.ru']['+7(495) 772-9590 *11830']
{'': 'Институт управления государственными ресурсами центр государственного сектора экономики', 'Профессор:': 'Институт образования Департамент образовательных программ', 'Директор центра:': 'Институт управления государственными ресурсами центр государственного сектора экономики'}
Абанокова Ксения Руслановна['kabanokova@hse.ru']['+7(495) 624-6240', '+7(495) 772-9590']
{'Младший научный сотрудник:': 'Институт социальной политики Центр семейной политики и качества жизни'}
Абашкин Василий Львович['vabashkin@hse.ru']['+7(495) 772-9590 * 11609']
{'': 'Институт статистических исследований и экономики знаний', 'Ведущий эксперт:': 'Институт

### XPath

In [10]:
from lxml import html 

tree = html.fromstring(page.content)

In [11]:
def position_xpath(i):
    pos = []
    position = {}
    dolzn = tree.xpath('//div/div[@class="post person"]['+str(i)+']/div/div/div/p[@class="with-indent7"]/span/text()')
    for i in range(len(dolzn)):
        dolzn[i] = dolzn[i].strip()
        if len(dolzn[i]) > 1:
            pos.append(dolzn[i])
    for i in range(1,len(pos)+1):
        p = tree.xpath('//div/div[@class="post person"]['+str(i)+']/div/div/div/p[@class="with-indent7"]/span/a[@class="link"]/text()')
        position[pos[i-1]]=p
    return position

In [12]:
def emails_xpath(i):
    mails = tree.xpath('//div/div[@class="post person"]['+str(i)+']/div/div[@class="l-extra small"]/a[@data-at]/attribute::data-at')
    for m in range(len(mails)):
        mail = (mails[m].replace("-at-", "@")).split('"')[1::2]
        mails[m] = "".join(mail)
    return mails

In [13]:
def names_xpath():
    name = []
    n = tree.xpath('//a[@class="link link_dark large b"]/text()')
    for i in range(len(n)):
        n[i] = n[i].strip()
        if len(n[i]) > 1:
            name.append(n[i])
    return name

In [14]:
def teachers_with_xpath():
    profs = []
    names = names_xpath()
    for i in range(1, len(names)+1):
        phone = tree.xpath('//div/div[@class="post person"]['+str(i)+']/div/div[@class="l-extra small"]/span/text()')  
        prof = Professor(names[i-1], emails_xpath(i), phone, position_xpath(i))
        profs.append(prof)
    return profs

In [15]:
profs_xpath = teachers_with_xpath()

for p in profs_xpath:
    print(p)

Абанкина Ирина Всеволодовна['abankinai@hse.ru']['+7(495) 772-9590 *22073']
{'Директор:': ['Институт управления государственными ресурсами', 'центр государственного сектора экономики', 'Институт образования', 'Департамент образовательных программ'], 'Профессор:': ['Институт образования', 'Департамент образовательных программ', 'Институт образования', 'Институт развития образования']}
Абанкина Татьяна Всеволодовна['abankinat@hse.ru']['+7(495) 772-9590 *11830']
{'Профессор:': ['Институт управления государственными ресурсами', 'центр государственного сектора экономики', 'Институт образования', 'Департамент образовательных программ'], 'Директор центра:': ['Институт образования', 'Департамент образовательных программ', 'Институт образования', 'Институт развития образования']}
Абанокова Ксения Руслановна['kabanokova@hse.ru']['+7(495) 624-6240', '+7(495) 772-9590']
{'Младший научный сотрудник:': ['Институт образования', 'Департамент образовательных программ', 'Институт образования', 'Институт 